In [1]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt

import numpy as np
import os
import pandas as pd
import xml.etree.ElementTree as ET

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

from sklearn.model_selection import train_test_split

from tensorflow.keras.preprocessing import image

from sklearn.metrics import roc_curve, auc, confusion_matrix, classification_report,precision_recall_curve
import seaborn as sns


In [2]:
sign_dict = {
  "trafficlight": 1,
  "stop": 2,
  "speedlimit": 3,
  "crosswalk": 4,
}

In [ ]:

# Path to image and annotation directories
images = "/Users/favourokpali/Documents/Semester 2/ECE9022-Assignments/FinalProject/output_images/"
annotations = "/Users/favourokpali/Documents/Semester 2/ECE9022-Assignments/FinalProject/annotations/"

# Initialize empty lists for image data and labels
X_data = []
Y_data = []


# Set the desired image size (e.g., 256x256)
image_size = (256, 256)

# Iterate through images and annotations
for image_name in os.listdir(images):
    image_root_name = image_name.split('.')[0]
    
    for annotation_name in os.listdir(annotations):
        annotation_root_name = annotation_name.split('.')[0]
        
        # Match image and annotation by root name
        if annotation_root_name == image_root_name:
            annotation_path = os.path.join(annotations, annotation_name)
            tree = ET.parse(annotation_path)
            root = tree.getroot()
            num_objects = len(root.findall("object"))

            if num_objects == 1:

                # Read and resize the image to a consistent shape (e.g., 256x256)
                img_path = os.path.join(images, image_name)
                try:
                    img = mpimg.imread(img_path)
                    img_resized = image.smart_resize(img, image_size)  # Resize image to 256x256
                    img_resized = np.ravel(img_resized)  # Flatten image data
                    X_data.append(img_resized)
                except Exception as e:
                    print(f"Error loading or resizing image {img_path}: {e}")
                    continue  # Skip this image if it fails to load or resize

                # Parse annotation data (ensure proper indexing based on your XML structure)
                y_value = root[4][0].text
                
                # Encode label based on the dictionary
                try:
                    annotation_encoded = sign_dict[str(y_value)]
                    Y_data.append(annotation_encoded)
                except KeyError:
                    print(f"Warning: Label {y_value} not found in sign_dict. Skipping.")
                    continue  # Skip this label if it is not found

# Convert to DataFrames
X_data = np.array(X_data)
Y_data = np.array(Y_data)


In [4]:
X_data = X_data.reshape((X_data.shape[0], 512, 512, 1)) 

Y_data = Y_data.flatten()

# Print unique values of Y_data to check the range
Y_data_encoded = to_categorical(Y_data, num_classes=5)

x_train, x_test, y_train, y_test = train_test_split(X_data, Y_data_encoded, test_size=0.2, random_state=42)


In [ ]:
# Build the CNN model
model = models.Sequential()

# Add a convolutional layer with 32 filters, 3x3 kernel, and ReLU activation
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(512, 512, 1)))  # Adjust input shape as necessary

# Add a max-pooling layer
model.add(layers.MaxPooling2D((2, 2)))

# Add another convolutional layer with 64 filters
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

# Add another max-pooling layer
model.add(layers.MaxPooling2D((2, 2)))

# Add a flatten layer to flatten the 2D matrix into 1D
model.add(layers.Flatten())

# Add a fully connected (dense) layer with 128 neurons and ReLU activation
model.add(layers.Dense(128, activation='relu'))

# Output layer with 4 neurons (one for each class) and softmax activation
model.add(layers.Dense(5, activation='softmax'))

# Compile the model with categorical cross-entropy loss, Adam optimizer, and accuracy metric
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(x_train, y_train, epochs=10, batch_size=64, validation_data=(x_test, y_test))
y_pred = model.predict(x_test)

# Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test)

print(f"Test Accuracy: {test_acc * 100:.2f}%")



/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 31s 4s/step - accuracy: 0.6981 - loss: 42.9343 - val_accuracy: 0.7339 - val_loss: 7.3867
Epoch 2/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 29s 4s/step - accuracy: 0.7519 - loss: 3.7243 - val_accuracy: 0.6774 - val_loss: 1.5723
Epoch 3/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 31s 4s/step - accuracy: 0.7616 - loss: 1.5942 - val_accuracy: 0.7500 - val_loss: 0.8412
Epoch 4/10


In [ ]:

# 1. Accuracy vs. Loss Plot
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()


# 3. Confusion Matrix
y_test_labels = np.argmax(y_test, axis=1)  # Convert one-hot to label format
y_pred_classes = np.argmax(y_pred, axis=1)  # If multi-class
cm = confusion_matrix(y_test_labels, y_pred_classes)
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=np.unique(y_test_labels), yticklabels=np.unique(y_test_labels))
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

# 4. Precision-Recall Curve
plt.figure(figsize=(6, 6))
for i in range(y_test.shape[1]):  # Loop through each class
    precision, recall, _ = precision_recall_curve(y_test[:, i], y_pred[:, i])
    plt.plot(recall, precision, label=f'Class {i}')
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.legend()
plt.title("Precision-Recall Curve")
plt.show()

